# Defining Params

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import seaborn as sns
import pandas as pd
from keras.models import model_from_json
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
batch_size = 32
img_height=80
img_width = 256
approx_fold_size = 1000

# Load Model

In [4]:
path = "/media/romulo/6237-3231/augmented_model_v2/"
model_definition_location = path+"/augmented_model_three_classes_unfrozen_layers_weighted_v2.json"
model_weights_location = path+"/augmented_model_three_classes_unfrozen_layers_weighted_v2.h5"
model_labels_location = path+"/augmented_model_three_classes_unfrozen_layers_weighted_v2_class_indices.json"
test_folder = "/media/romulo/6237-3231/soundflux_stress_testing/spectrograms"

In [5]:
with open(model_definition_location, 'r') as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(model_weights_location)
with open(model_labels_location, 'r') as json_file:
    label_map = json.load(json_file)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


# Evaluate Model

In [6]:
datagen = ImageDataGenerator(rescale=1./255,
                            #rotation_range=10,
                            width_shift_range=0.1,
                            #horizontal_flip=True,
                            #vertical_flip=True,
                            fill_mode='nearest')

In [7]:
test_generator = datagen.flow_from_directory(test_folder,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=False, 
                                          seed=7)

Found 3 images belonging to 1 classes.


In [8]:
steps = np.ceil(len(test_generator.classes)/batch_size)

In [9]:
predictions = model.predict_generator(test_generator,
                                      steps = steps,
                              verbose=True)

1/1 [==============================] - 0s 366ms/step


# Results

In [10]:
y_pred = np.argmax(predictions, axis=1)

In [11]:
y_pred

array([1, 1, 2])

In [12]:
mapped_results = []
for i in y_pred:
    r_map = {}
    for k, v in label_map.items():
        if i == v:
            mapped_results.append(str(k))
            break

In [13]:
mapped_results

['falling_object', 'falling_object', 'overlay_noise']